In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve, auc, classification_report
from sklearn import metrics
import keras
from keras import Sequential, regularizers
from keras.optimizers import SGD, RMSprop, Adam
from keras.layers import Dense, Dropout, Activation
from keras.wrappers.scikit_learn import KerasRegressor
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder 
sns.set_style("whitegrid")  # Set style of seaborn
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
import math
from sklearn.preprocessing import MinMaxScaler

In [ ]:
adonis=pd.read_csv('../input/top-personality-dataset/2018-personality-data.csv')
##adonis['userid']=adonis['useri']
adonis.columns=adonis.columns.str.lstrip()#columns name contains empty spaces
adonis.head()

In [ ]:
adonis.info()

In [ ]:
socratic=pd.read_csv('../input/top-personality-dataset/2018_ratings.csv')
socratic.columns=socratic.columns.str.lstrip()#column names contains empty space
socratic.head()

In [ ]:
df_num = yaga.select_dtypes(include=[np.float, np.int])
df_num

In [ ]:
#Create a new dataframe with only the 'Close' column
data = adonis.filter(['emotional_stability'])
#Converting the dataframe to a numpy array
dataset = data.values
#Get /Compute the number of rows to train the model on
training_data_len = math.ceil( len(dataset) *.8)

In [ ]:
#Scale the all of the data to be values between 0 and 1 
scaler = MinMaxScaler(feature_range=(0, 1)) 
scaled_data = scaler.fit_transform(dataset)

In [ ]:
#Create the scaled training data set 
train_data = scaled_data[0:training_data_len  , : ]
#Split the data into x_train and y_train data sets
x_train=[]
y_train = []
for i in range(60,len(train_data)):
    x_train.append(train_data[i-60:i,0])
    y_train.append(train_data[i,0])

In [ ]:
#Convert x_train and y_train to numpy arrays
x_train, y_train = np.array(x_train), np.array(y_train)

In [ ]:
#Reshape the data into the shape accepted by the LSTM
x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))

In [ ]:
#Build the LSTM network model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True,input_shape=(x_train.shape[1],1)))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dense(units=25))
model.add(Dense(units=1))

#Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
#Train the model
model.fit(x_train, y_train, batch_size=32, epochs=20)

In [ ]:
#Test data set
test_data = scaled_data[training_data_len - 60: , : ]
#Create the x_test and y_test data sets
x_test = []
y_test =  dataset[training_data_len : , : ] #Get all of the rows from index 1603 to the rest and all of the columns (in this case it's only column 'Close'), so 2003 - 1603 = 400 rows of data
for i in range(60,len(test_data)):
    x_test.append(test_data[i-60:i,0])

In [ ]:
#Convert x_test to a numpy array 
x_test = np.array(x_test)

In [ ]:
#Reshape the data into the shape accepted by the LSTM
x_test = np.reshape(x_test, (x_test.shape[0],x_test.shape[1],1))

In [ ]:
#Getting the models predicted price values
predictions = model.predict(x_test) 
predictions = scaler.inverse_transform(predictions)#Undo scaling

In [ ]:
#Calculate/Get the value of RMSE
rmse=np.sqrt(np.mean(((predictions- y_test)**2)))
rmse

In [ ]:
#Plot/Create the data for the graph
train = data[:training_data_len]
valid = data[training_data_len:]
valid['Predictions'] = predictions
#Visualize the data
plt.figure(figsize=(16,8))
plt.title('Model')
plt.xlabel('emotional_stability', fontsize=18)
plt.ylabel('Y label', fontsize=18)
plt.plot(train['emotional_stability'])
plt.plot(valid[['emotional_stability', 'Predictions']])
plt.legend(['Train', 'Val', 'Predictions'], loc='lower right')
plt.show()

In [ ]:
#Show the valid and predicted prices
valid

##DNN

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
adonis

In [ ]:
avg = adonis['emotional_stability'] + adonis['agreeableness']+adonis['extraversion']+adonis['conscientiousness']+adonis['openness']
adonis["Temprament"]=avg/5

In [ ]:
adonis = adonis[['openness','agreeableness','emotional_stability','conscientiousness','extraversion','Temprament']]

In [ ]:
adonis

In [ ]:
from sklearn.model_selection import train_test_split
X = adonis.iloc[:, :-1].values
y = adonis.iloc[:, -1].values

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(X,y,random_state = 0,test_size = 0.2)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [ ]:
model = keras.Sequential([ 
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1)
    ])
model.compile(optimizer='rmsprop', loss='mse', metrics=['mae']) # observe the loss and metrics

In [ ]:
history = model.fit(x_train, y_train, epochs=30, batch_size=16, validation_data=(x_test, y_test))

In [ ]:
[mse, mae] = model.evaluate(x_test, y_test) 

In [ ]:
ypred = model.predict(x_test)
print('Temarament = {} and Predicted Temparament = {}'.format(y_test[1], ypred[1]))

In [ ]:
# We create a fucntion to make it easy for multiple calls
def build_model():   
    model = keras.Sequential([ 
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae']) # observe the loss and metrics
    return model

In [ ]:
# Let us visit K-fold validation
k = 4
num_val_samples = len(x_train) // k
num_epochs = 100
all_scores = []
for i in range(k):
    print('processing fold #%d' % i)
    val_data = x_train[i * num_val_samples: (i + 1) * num_val_samples] 
    val_labels = y_train[i * num_val_samples: (i + 1) * num_val_samples]
    partial_train_data = np.concatenate( 
        [x_train[:i * num_val_samples],
         x_train[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_labels = np.concatenate(
        [y_train[:i * num_val_samples],
         y_train[(i + 1) * num_val_samples:]],
        axis=0)
    
model = build_model() 
model.fit(partial_train_data, partial_train_labels, 
          epochs=num_epochs, batch_size=1, verbose=0)
val_mse, val_mae = model.evaluate(val_data, val_labels, verbose=0) 
all_scores.append(val_mae)

In [ ]:
print(all_scores)
print(np.mean(all_scores))

In [ ]:
model = build_model() 
model.fit(x_train, y_train, epochs=16, batch_size=16, verbose=0)
test_mse_score, test_mae_score = model.evaluate(x_test, y_test)
print(test_mse_score, test_mae_score)

In [ ]:
# plot loss metrics
ax1 = plt.plot(history.history["loss"])
ax2 = plt.plot(history.history["val_loss"])
plt.legend(["loss", "val_loss"])

In [ ]:
# plot mae metrics
ax1 = plt.plot(history.history["mae"])
ax2 = plt.plot(history.history["val_mae"])
plt.legend(["mae", "val_mae"])

**Case 2- Employee Churn Rate**

In [ ]:
import numpy as np 
import pandas as pd

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.models import load_model
from tensorflow.keras.utils import plot_model

import math

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df=pd.read_csv('../input/human-resource-analytics-dataset/hr_data_preprocessed.csv')
df.head()

In [ ]:
df.shape

In [ ]:
#df.isnull().sum()

In [ ]:
#df = df.drop('is_smoker', axis=1)

In [ ]:
#df['average_montly_hours']=df['average_montly_hours'].fillna(df['average_montly_hours'].mean())
#df['time_spend_company']=df['time_spend_company'].fillna(df['time_spend_company'].mean())

In [ ]:
#df.isnull().sum()

In [ ]:
#fig, ax = plt.subplots(figsize=(24,12)) # figsize
#sns.heatmap(df.corr(),annot=True,linewidths=.5,fmt='.1g',cmap= 'coolwarm')

In [ ]:
y = df['left']
x = df.drop('left', axis=1)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33)
print('%s , %s , %s , %s' % (len(x_train), len(y_train), len(x_test), len(y_test)))

In [ ]:
def build_dnn1():
    dnn = Sequential()
    dnn.add(Dense(units=64, kernel_initializer='uniform', activation='relu', input_dim=20))
    dnn.add(Dense(units=32, kernel_initializer='uniform', activation='relu'))
    dnn.add(Dense(units=16, kernel_initializer='uniform', activation='relu'))
    dnn.add(Dense(units=8, kernel_initializer='uniform', activation='relu'))
    dnn.add(Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))
    dnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return dnn

In [ ]:
model = build_dnn1()
model.fit(x_train, y_train, 
          epochs=30, batch_size=16, verbose=0)
print(model.evaluate(x_train, y_train))

In [ ]:
# Adding regularization layers i.e Dropot layers
def build_dnn2():
    dnnmodel = Sequential()
    dnnmodel.add(Dense(64, activation='relu', kernel_initializer='uniform'))
    dnnmodel.add(Dropout(0.5))
    dnnmodel.add(Dense(32, activation='relu', kernel_initializer='uniform'))
    dnnmodel.add(Dropout(0.5))
    dnnmodel.add(Dense(16, activation='relu', kernel_initializer='uniform'))
    dnnmodel.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
    dnnmodel.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return dnnmodel

In [ ]:
model1 = build_dnn2()
model1.fit(x_train, y_train, 
          epochs=30, batch_size=16, verbose=0)
print(model1.evaluate(x_train, y_train))

In [ ]:
# Using three-fold validation to check how model would perform on useen data
k = 3
num_val_samples = len(x_train) // k
all_scores = []
for i in range(k):
    print('processing fold #%d' % i)
    val_data = x_train[i * num_val_samples: (i + 1) * num_val_samples] 
    val_labels = y_train[i * num_val_samples: (i + 1) * num_val_samples]
    partial_train_data = np.concatenate( 
        [x_train[:i * num_val_samples],
         x_train[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_labels = np.concatenate(
        [y_train[:i * num_val_samples],
         y_train[(i + 1) * num_val_samples:]],
        axis=0)

In [ ]:
dnn1 = build_dnn1()
dnn2 = build_dnn2()
dnn1.fit(partial_train_data, partial_train_labels, 
          epochs=30, batch_size=16, verbose=0)
dnn2.fit(partial_train_data, partial_train_labels, 
          epochs=30, batch_size=16, verbose=0)
scores_val_dnn1 = dnn1.evaluate(val_data, val_labels, verbose=0)
scores_val_dnn2 = dnn2.evaluate(val_data, val_labels, verbose=0)
print("Scores_dnn1: \n",scores_val_dnn1[1]*100)
print("Scores_dnn2: \n",scores_val_dnn2[1]*100)

In [ ]:
plot_model(dnn1, to_file='dnn1.png', show_shapes=True)

In [ ]:
plot_model(dnn2, to_file='dnn2.png', show_shapes=True)

In [ ]:
pred = dnn1.predict(x_test)
y_pred = [round(i[0]) for i in pred]
accuracy_score(y_test, y_pred)

In [ ]:
pred1 = dnn2.predict(x_test)
y_pred1 = [round(i[0]) for i in pred1]
accuracy_score(y_test, y_pred1)